Spotify คือหนึ่งในการบริการฟังเพลงที่ใหญ่ที่สุดในโลก ซึ่งจากสถิติมีคนฟังเพลงราว 30 ล้านเพลงต่อวัน โดยเพลงที่ติดอันดับตั้งแต่ปี 2010 ถึง 2019 ในปัจจุบันก็ยังคงถูกฟังเหมือนเดิม 

**การนำเข้าข้อมูล โดย import และประกาศค่าตัวแปรต่างๆ**

In [ ]:

#การนำเข้าข้อมูล โดย import และประกาสค่าตัวแปรต่างๆ
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.interactiveshell import InteractiveShell
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import silhouette_score
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

**1. Question or problem definition**

เนื่องจากในปัจจุบันมีแนวเพลงเกิดขึ้นมากมายและมีแนวโน้มว่าจะสูงขึ้นเรื่อยๆ นักแต่งเพลงส่วนใหญ่จึงจำเป็นต้องใช้ข้อมูลจากผลการวิจัย (Research) หรือผลรวบรวมข้อมูลอันดับเพลงต่างๆเพื่อประกอบการทำเพลง ข้อมูลนี้จึงเป็นสิ่งที่สำคัญมากสำหรับนักแต่งเพลง เช่น แนวเพลง จังหวะดนตรี หรือแม้ประทั่งชื่อศิลปินที่เป็นที่นิยม เพื่อประมวลว่าแท้จริงแล้วคนฟังชอบฟังสไตล์การร้องแบบไหนศิลปินคนไหน เพื่อทำการแต่งเพลงใหม่ๆให้เป็นที่นิยม 


**2. Acquire training and testing data.
**

จากปัญหาที่พบเราจึงนำผลลัพธ์ของข้อมูลเพลงที่ติดอันดับทั้งหมดจากปี 2010 - 2019 มาใช้ในการทำงาน โดยการนำเข้า(import) ข้อมูล ดังนี้

หมวดหมู่ที่เขียนด้วยข้อความเพื่อใช้ในการนำข้อมูลเข้า (input)
- Title : ชื่อเพลงที่ติดอันดับ
- Artist : ชื่อศิลปินที่ร้องเพลงนี้
- Top Genre : แนวเพลงที่ติดอันดับ

หมวดหมู่ที่เขียนด้วยตัวเลขเพื่อใช้ในการนำข้อมูลเข้า (input)
- Year : ปีที่ติดอันดับ (หน่วยเป็น “ปี”)
- Beats.Per.Minute (bpm) : จังหวะดนตรีของเพลง (“หน่วยเป็น bpm”)
- Energy : พลังของเพลง หรือความพุ่งของเพลงโดยดูจากความเร็ว ความดัง (มีหน่วยเป็น “จูลน์ (J)”)
- Danceability : อัตราการเต้นของเพลง โดยดูจาก tempo, การย้ำจังหวะ, ความแข็งแรงของ beat 
- Loudness : ความดังในหน่วย decibel  ไม่ใช่ความดังแบบ popular ซึ่งจะเป็นค่าเฉลี่ยตลอดทั้งเพลง (“หน่วยเป็น dB”)
- Liveness : ความสดของดนตรี ถ้ามีค่าสูงก็แสดงว่า Liveness มีค่ามากเท่านั้น
- Valence : ความจุของเพลง
- Duration : ความสั้น-ยาวของดนตรี (มีหน่วยเป็น ”วินาที (s)”)
- Acousticness : อารมณ์ของเพลง
- Speechiness : จับเฉพาะเสียงคนพูด หรือ Rap
- popularity : ความนิยมของเพลงเมื่อเทียบกับจำนวนประชากร
- Clusters คือ ผลของข้อมูลเพลงที่ได้ทำการวิเคราะห์ออกมา ซึ่งถูกแบ่งออกเป็น 3 กลุ่ม

In [ ]:
# การนำข้อมูลเข้า โดยใช้ไฟล์ top10s.csv
filename='/kaggle/input/top-spotify-songs-from-20102019-by-year/top10s.csv'
df=pd.read_csv(filename,encoding='ISO-8859-1')
# การแสดงข้อมูลทั้งหมด 603 ข้อมูล ที่เป็นเพลงฮิตตลอด 10 ปี แต่เป็นการขึ้นต้นอันดับด้วย 0 จนถึง 602
df.head(603) 


**3. Wrangle, prepare, cleanse the data.**

เป็นการตรวจสอบข้อมูลว่ามีข้อมูลที่ผิดพลาดหรือเสียหายอย่างไรบ้าง เพื่อทำการเคลียร์ข้อมูลหรือนำข้อมูลที่ไม่ได้ใช้ออก

การทำขั้นตอนนี้ต้องอาศัยความแม่นยำและระเอียดอ่อนเพื่อให้ได้รับข้อมูลที่ถูกต้อง 100% และ มีความผิดพลาด/คลาดเคลื่อนน้อยที่สุดจึงต้องมีการตรวจสอบหลายครั้งเพื่อให่มั่นใจ

In [ ]:
#เปลี่ยนชื่อ columns จาก 'Unnamed: 0'เป็น 'id'
df = df.rename(columns = {'Unnamed: 0': 'id'})
df.head()

การนำตารางมาใช้ต้องอาศัยการเคลียร์ข้อมูลก่อนทั้งหมด เพื่อให้ค่าทั้งหมดออกมาถูกต้องที่สุด ไม่คลาดเคลื่อน

ดังนั้นเราจึงทำการตรวจสอบผล โดยค่าที่ขึ้น false จะมีความหมายว่าข้อมูลถูกเคลียร์เรียบร้อยแล้วสามารถนำมาใช้งานต่อได้

In [ ]:
#จากการทำการตรวจสอบถ้าข้อมูลขึ้น false จะแสดงได้ว่าข้อมูลถูกเคลียร์เรียบร้อยแล้วสามารถนำข้อมูลมาใช้ได้ต่อ
df.isnull().any()

ทำการตรวจสอบข้อมูลหลังทำการเคลียร์ข้อมูลเรียบร้อยแล้ว โดย 603,15 ที่แสดงบนตารางมีความหมายดังนี้

 603 คอลัมน์ เป็นจำนวนของเพลงที่ติดอันดับตั้งแต่ปี 2010-2019 ทั้งหมดจากในตาราง
 
 15 คอลัมน์ คือ หมวดหมู่ของข้อมูลที่มีการเก็บข้อมูล

In [ ]:
#จำนวนข้อมูลของ columns ก่อน drop ข้อมูลทิ้ง
df = df.drop_duplicates()
df.shape

In [ ]:
#การ drop ข้อมูลทิ้ง ใน columns id
df = df.drop(['id'], axis=1)
df.head()

เป็นการแสดงผลว่า
1. มีแนวเพลงไหนบ้างที่ติดอันดับ (โดยแสดงผลเป็นตัวอักษร)
2. เพลงในแนวนั้นมีติดกี่จำนวนบ้าง (โดยผลจะแสดงเป็นตัวเลข) มีหน่วยเป็น "เพลง"

ซึ่งการแสดงแนวเพลงที่ติดอันดับในปี 2010 - 2019 จำนวน 5 อันดับแรกนี้เป็นส่วนหนึ่งที่สำคัญสำหรับการนับสถิติเพื่อทำการวิจัย (Research)

In [ ]:
#การนับจำนวน เพลงที่คนนิยมในช่วง 10 ปี ว่าแต่ละแนวเพลงจำนวนเท่าไหร่
df['top genre'].value_counts().head()


เป็นการแสดงผลว่า
1. มีศิลปินคนไหนบ้างที่ติดอันดับ (โดยแสดงผลเป็นตัวอักษร)
2. ศิลปินคนนั้นมีติดกี่จำนวนบ้าง (โดยผลจะแสดงเป็นตัวเลข) มีหน่วยเป็น "ครั้ง"

ซึ่งการแสดงศิลปินที่ติดอันดับในปี 2010 - 2019 จำนวน 5 อันดับแรกนี้เป็นส่วนหนึ่งที่สำคัญสำหรับการนับสถิติเพื่อทำการวิจัย (Research)

In [ ]:
#การนับจำนวน ศิลปินที่ร้องเพลงที่คนนิยมในช่วง 10 ปี ว่าแต่คนร้องเพลงจำนวนเท่าไหร่
df['artist'].value_counts().head()

เป็นการแสดงผลว่า
1. มีเพลงชื่อซ้ำเพลงไหนบ้างที่ติดอันดับ (โดยแสดงผลเป็นตัวอักษร)
2. เพลงที่มีชื่อซ้ำที่ติดอันดับมีกี่จำนวนกี่เพลงบ้าง (โดยผลจะแสดงเป็นตัวเลข) มีหน่วยเป็น "เพลง"

ซึ่งการแสดงชื่อเพลงที่ตั้งซ้ำกันเป็นอีกหนึ่งปัจจัยสำคัญสำหรับนักแต่งเพลงในการตั้งชื่อเพลงที่คล้ายคลึงหรือชื่อเดียวกัน เพื่อให้เป็นที่สนใจมากขึ้น

In [ ]:
#การนับจำนวน ชื่อเพลงที่คนนิยมในช่วง 10 ปี ว่ามีเพลงที่ซ้ำกันกี่เพลง
df['title'].value_counts().head()

แสดงผลจากการค้นหาเพลงที่มีชื่อว่า 'Company' ของศิลปิน 'Justin Bieber' ที่ติดอันดับภายในปี 2010 - 2019 โดยผลที่ออกมาพบว่ามีการติดอันดับ 2 ครั้ง คือปี 2015 และ 2016

In [ ]:
#แสดงค่าว่าตอนนี้เพลง Company และ deduplicate rows ที่ค่าซ้ำกัน 
df[df.title == 'Company']

การแสดงค่าแนวเพลงทั้งหมดที่ติดอันดับในปี 2010-2019 โดยจะแสดงผลเป็นตัวอักษร และ แนวเพลงนั้นมีกี่เพลง โดยจะแสดงผลเป็นตัวเลข หน่วยเป็น "เพลง"

In [ ]:
#นับค่าแนวเพลงทั้งหมด ว่าแต่ละแนวเพลงมีกี่เพลง
df['top genre'].value_counts()

**4. Analyze, identify patterns, and explore the data.**

ทำการวิเคราะห์และสำรวจข้อมูลที่จะนำไปตอบคำถามเรื่องการแต่งเพลง ควรมีอะไรบ้าง

ซึ่งสิ่งที่ควรมี คือ BPM, Energy, Danceability, Loudness, Valence, Acousticness Speechiness และ Popularity

ซึ่งนำข้อมูลที่ทราบมาทำดังนี้

ทำการทิ้ง (Drop) ข้อมูลที่เป็น Top Genre , Artist , Year , Title , Population ออกจาก Columns



In [ ]:
#ทำการ drop หรือทิ้ง  columns  year, title , pop , artist , top genre จาก data df และแทนค่า ด้วย data = yearless_df
#จะได้ตารางที่เหลือแต่ข้อมูลที่เราจะนำไปเทรนโมเดล
yearless_df = df.drop(['year', 'title', 'pop','artist' ,'top genre'], axis=1)
yearless_df.drop_duplicates()

**5. Model, predict and solve the problem.**

จากการวิเคราะห์ข้อมูลเรียบร้อยแล้ว เราจะใช้วิธี clustering หรือการรวมข้อมูลจัดเป็นกลุ่ม โดยจะมีการแบ่งเป็น 10 กลุ่ม

สาเหตุที่ต้องมีการแบ่งออกเป็นหลายประเภทนั้นเพราะคนฟังเพลงมีหลากหลายประเภท ซึ่งจำเป็นต้องเทียบจากคนหลายประเภทเพื่อให้ทราบถึงแนวเพลงที่หลากหลายมากขึ้นและกว้างมากขึ้น







In [ ]:
#การทำ clustering ด้วย KMeans
#k = กลุ่มที่เราต้องการจะจับ
#sse = ระยะห่างของ k หรือความห่างของแต่ละกลุ่ม
from sklearn.cluster import KMeans

sse = {}
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, max_iter=1000).fit(yearless_df)
    yearless_df["clusters"] = kmeans.labels_
    #print(data["clusters"])
    sse[k] = kmeans.inertia_ # Inertia: Sum of distances of samples to their closest cluster center
plt.figure()
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel("Number of cluster")
plt.ylabel("SSE")
plt.show()


K-means (k) คือ เป็นการเปรียบเทียบความคล้ายของข้อมูล กับจุดศูนย์กลางของแต่ละ Cluster อยู่ในกลุ่มของ Unsupervised Learning 

Error Sum of Squares (SSE) คือ การหาผลรวมกำลังสองความคลาดเคลื่อน

ใช้กระบวนการการแบ่งกลุ่มของ kmeans ด้วยหลักการของคอมพิวเตอร์ ออกมาทั้งหมด 10 กลุ่มตัวอย่างดังนี้

In [ ]:
#การทำ kmeans โดยแบ่งข้อมูลเป็น 10 กลุ่ม กลุ่มที่ 0-9
kmeans = KMeans(n_clusters=10, max_iter=1000).fit(yearless_df)
df["clusters"] = kmeans.labels_
df.groupby(by=['clusters']).mean()

หน่วยของแต่ละ column
- Year : ปีที่ติดอันดับ (หน่วยเป็น “ปี”)
- Beats.Per.Minute  (bpm) : จังหวะดนตรีของเพลง (“หน่วยเป็น bpm”)
- Energy : พลังของเพลง หรือความพุ่งของเพลงโดยดูจากความเร็ว ความดัง (มีหน่วยเป็น “จูลน์ (J)”)
- Danceability : อัตราการเต้นของเพลง โดยดูจาก tempo, การย้ำจังหวะ, ความแข็งแรงของ beat 
- Loudness : ความดังในหน่วย decibel  ไม่ใช่ความดังแบบ popular ซึ่งจะเป็นค่าเฉลี่ยตลอดทั้งเพลง (“หน่วยเป็น dB”)
- Liveness : ความสดของดนตรี ถ้ามีค่าสูงก็แสดงว่า Liveness มีค่ามากเท่านั้น
- Valence : ความจุของเพลง
- Duration : ความสั้น-ยาวของดนตรี (มีหน่วยเป็น ”วินาที (s)”)
- Acousticness : อารมณ์ของเพลง
- Speechiness : จับเฉพาะเสียงคนพูด หรือ Rap
- popularity : ความนิยมของเพลงเมื่อเทียบกับจำนวนประชากร

**6.Visualize, report, and present the problem solving steps and final solution.**

จากที่ทำการวิเคราะห์ข้อมูลเสร็จเรียบร้อยแล้ว จึงทำการสรุปผลข้อมูลออกมา 
เพื่อแบ่งคนออกเป็น 10กลุ่ม จากนั้นเราจึงทำการเพิ่มคอลัมภ์ Clustering เพื่อให้รู้ว่าเพลงนั้นอยู่ในกลุ่มใด

ซึ่งข้อมูลนี้จะเป็นประโยชน์ในการนำมาประยุกต์ใช้สำหรับนักแต่งเพลง เนื่องจากคนฟังนั้นมีหลากหลายประเภท 
ซึ่งจำเป็นต้องเทียบจากคนหลายประเภทเพื่อให้ทราบถึงแนวเพลงที่หลากหลายมากขึ้นและกว้างมากขึ้น

In [ ]:
#การแสดงข้อมูลทั้งหมด และบ่งบอกว่าแต่ละเพลง จัดอยู่ในกลุ่มที่เท่าไหร่ ใน clusters
df.head(603)

เป็นการแสดงแบบ PairPlot Graph เพื่องานต่อการแสดงข้อมูลที่หลากหลายและครบถ้วน
โดยจะมีการแบ่งเพลงเป็นสีที่ต่างกัน และหมวดหมู่จะถูกไล่ทีละหมวดเพื่อทำให้กราฟถูกแสดงอย่างครบถ้วน

In [ ]:
#การแสดงข้อมูลโดย pairplot เพื่อแสดงข้อมูลทั้งหมด ว่าสีนี้อยู่กลุ่มอะไร และกราฟมีการพล็อตจุดเป็นสีช่วงไหน
sns.pairplot(df, hue="clusters")

ซึ่งจากการวิเคราะห์และทำการแบ่งออกเป็น 10 กลุ่ม (Cluster) โดยที่ 

Count คือ ข้อมูล (data) ของเพลงทั้งหมด ซึ่งมีจำนวน 603 เพลง (หน่วยเป็น “เพลง”)

Clusters คือ ผลของข้อมูลเพลงที่ได้ทำการวิเคราะห์ออกมา ซึ่งถูกแบ่งออกเป็น 10 กลุ่ม โดยกราฟนี้แสดงให้เห็นถึงในแต่ละกลุ่มมีข้อมูลของเพลงอยู่จำนวนเท่าไหร่

ซึ่งกราฟนี้มีการใช้ Count plot เพื่อง่ายต่อการแสดงข้อมูล

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(df['clusters'])
plt.show()

ซึ่งจากการวิเคราะห์และทำการแบ่งออกเป็น 10 กลุ่ม (Cluster) โดยที่

Beats.Per.Minute  (BPM) คือ การแสดงจังหวะดนตรีของเพลง (มีหน่วยเป็น “bpm”)

Clusters คือ ผลของข้อมูลเพลงที่ได้ทำการวิเคราะห์ออกมา ซึ่งถูกแบ่งออกเป็น 10 กลุ่ม

โดยกราฟนี้แสดงค่า Beats.Per.Munite (BPM) ในแต่ละช่วงโดยแบ่งตามกลุ่ม เพื่อให้รู้ค่า Beats.Per.Munite (BPM) ในแต่ละเพลง

กราฟนี้มีการใช้ Cat plot เพื่อง่ายต่อการแสดงข้อมูล

In [ ]:
sns.catplot(x="clusters", y="bpm", data=df)

ซึ่งจากการวิเคราะห์และทำการแบ่งออกเป็น 10 กลุ่ม (Cluster) โดยที่

Energy คือ พลังของเพลง หรือความพุ่งของเพลงโดยดูจากความเร็ว ความดัง (มีหน่วยเป็น “จูลน์”)

Clusters คือ ผลของข้อมูลเพลงที่ได้ทำการวิเคราะห์ออกมา ซึ่งถูกแบ่งออกเป็น 10 กลุ่ม

โดยกราฟนี้แสดงค่า Energy ในแต่ละช่วงโดยแบ่งตามกลุ่ม เพื่อให้รู้ค่า Energy ในแต่ละเพลง

กราฟนี้มีการใช้ Cat plot เพื่อง่ายต่อการแสดงข้อมูล

In [ ]:
sns.catplot(x = "clusters", y  ="nrgy" , data=df)


ซึ่งจากการวิเคราะห์และทำการแบ่งออกเป็น 10 กลุ่ม (Cluster) โดยที่

Danceability คือ อัตราการเต้นของเพลง โดยดูจาก tempo, การย้ำจังหวะ, ความแข็งแรงของ beat

Clusters คือ ผลของข้อมูลเพลงที่ได้ทำการวิเคราะห์ออกมา ซึ่งถูกแบ่งออกเป็น 10 กลุ่ม

โดยกราฟนี้แสดงค่า Danceabilityในแต่ละช่วงโดยแบ่งตามกลุ่ม เพื่อให้รู้ค่า Danceability ในแต่ละเพลง

กราฟนี้มีการใช้ Cat plot เพื่อง่ายต่อการแสดงข้อมูล

In [ ]:
sns.catplot(x="clusters",y="dnce",data=df)

ซึ่งจากการวิเคราะห์และทำการแบ่งออกเป็น 10 กลุ่ม (Cluster) โดยที่

Loudness คือ ความดังในหน่วย decibel ไม่ใช่ความดังแบบ popular ซึ่งจะเป็นค่าเฉลี่ยตลอดทั้งเพลง (“หน่วยเป็น dB”)

Clusters คือ ผลของข้อมูลเพลงที่ได้ทำการวิเคราะห์ออกมา ซึ่งถูกแบ่งออกเป็น 10 กลุ่ม

โดยกราฟนี้แสดงค่า Loudnessในแต่ละช่วงโดยแบ่งตามกลุ่ม เพื่อให้รู้ค่า Loudness ในแต่ละเพลง

กราฟนี้มีการใช้ Cat plot เพื่อง่ายต่อการแสดงข้อมูล โดยมีการไล่ตั้งแต่ 0 จนติดลบไปเรื่อยๆ ซึ่งเป็นค่าโดยปกติของ Loudness ในหน่วย dB

In [ ]:
sns.catplot(x="clusters", y="dB", data=df)

ซึ่งจากการวิเคราะห์และทำการแบ่งออกเป็น 10 กลุ่ม (Cluster) โดยที่

Liveness คือ ความสดของดนตรี ถ้ามีค่าสูงก็แสดงว่า Liveness มีค่ามากเท่านั้น

Clusters คือ ผลของข้อมูลเพลงที่ได้ทำการวิเคราะห์ออกมา ซึ่งถูกแบ่งออกเป็น 10 กลุ่ม

โดยกราฟนี้แสดงค่า Liveness ในแต่ละช่วงโดยแบ่งตามกลุ่ม เพื่อให้รู้ค่า Liveness ในแต่ละเพลง

กราฟนี้มีการใช้ Cat plot เพื่อง่ายต่อการแสดงข้อมูล

In [ ]:
sns.catplot(x="clusters", y="live", data=df)

ซึ่งจากการวิเคราะห์และทำการแบ่งออกเป็น 10 กลุ่ม (Cluster) โดยที่

Valence คือ ความจุของเพลง

Clusters คือ ผลของข้อมูลเพลงที่ได้ทำการวิเคราะห์ออกมา ซึ่งถูกแบ่งออกเป็น 10 กลุ่ม

โดยกราฟนี้แสดงค่า Valence ในแต่ละช่วงโดยแบ่งตามกลุ่ม เพื่อให้รู้ค่า Valence ในแต่ละเพลง

กราฟนี้มีการใช้ Cat plot เพื่อง่ายต่อการแสดงข้อมูล

In [ ]:
sns.catplot(x="clusters", y="val", data=df)

ซึ่งจากการวิเคราะห์และทำการแบ่งออกเป็น 10 กลุ่ม (Cluster) โดยที่

Duration คือ ความสั้น-ยาวของดนตรี (มีหน่วยเป็น ”วินาที (s)”)

Clusters คือ ผลของข้อมูลเพลงที่ได้ทำการวิเคราะห์ออกมา ซึ่งถูกแบ่งออกเป็น 10 กลุ่ม

โดยกราฟนี้แสดงค่า Duration ในแต่ละช่วงโดยแบ่งตามกลุ่ม เพื่อให้รู้ค่า Duration ในแต่ละเพลง

กราฟนี้มีการใช้ Cat plot เพื่อง่ายต่อการแสดงข้อมูล

In [ ]:
sns.catplot(x="clusters", y="dur", data=df)

ซึ่งจากการวิเคราะห์และทำการแบ่งออกเป็น 10 กลุ่ม (Cluster) โดยที่

Acousticness คือ อารมณ์ของเพลง

Clusters คือ ผลของข้อมูลเพลงที่ได้ทำการวิเคราะห์ออกมา ซึ่งถูกแบ่งออกเป็น 10 กลุ่ม

โดยกราฟนี้แสดงค่า Acousticness ในแต่ละช่วงโดยแบ่งตามกลุ่ม เพื่อให้รู้ค่า Acousticness ในแต่ละเพลง

กราฟนี้มีการใช้ Cat plot เพื่อง่ายต่อการแสดงข้อมูล

In [ ]:
sns.catplot(x="clusters", y="acous", data=df)

ซึ่งจากการวิเคราะห์และทำการแบ่งออกเป็น 10 กลุ่ม (Cluster) โดยที่

Speechiness คือ จับเฉพาะเสียงคนพูด หรือ Rap

Clusters คือ ผลของข้อมูลเพลงที่ได้ทำการวิเคราะห์ออกมา ซึ่งถูกแบ่งออกเป็น 10 กลุ่ม

โดยกราฟนี้แสดงค่า Speechiness ในแต่ละช่วงโดยแบ่งตามกลุ่ม เพื่อให้รู้ค่า Speechiness ในแต่ละเพลง

กราฟนี้มีการใช้ Cat plot เพื่อง่ายต่อการแสดงข้อมูล

In [ ]:
sns.catplot(x="clusters", y="spch", data=df)

ซึ่งบางกลุ่มค่าใกล้เคียงกันมากเราจึงใช้ การยกตัวอย่างจากกลุ่มทั้งหมด 10 กลุ่ม ดังนี้

ประเภทที่ 1 

Beats.Per.Minute (bpm) มีความเร็วมากที่สุด
Energy (ความแรง) ปานกลาง
Danceability (อัตราการเต้น) น้อย
Loudness หรือ ความดัง มีระดับปกติ
Liveness (ความสดของดนตรี) มีค่าสูง
Valence (ความจุเพลง) ปานกลาง
Duration (ความยาวเพลง) อยู่ที่ประมาณ 3.6 นาที
Acousticness (อารมณ์ของเพลง) เป็นเพลงร้องปกติ แต่มีความเร็วเพลงมาก
Speechiness (การร้องของเพลง) ค่อนข้างเยอะ

ประเภทที่ 2

Beats.Per.Minute (bpm) มีจังหวะที่เร็ว
Energy (ความแรง) ปานกลาง
Danceability (อัตราการเต้น) ปานกลาง
Loudness (ความดังในหน่วย dB) ปานกลาง
Liveness (ความสดของดนตรี) มีค่าปกติ
Valence (ความจุเพลง) น้อย
Duration (ความยาวเพลง ) อยู่ที่ประมาณ 3.5 นาที
Acousticness (อารมณ์ของเพลง) เป็นเพลงร้องปกติ และ มีความเร็วอยู่บ้าง
Speechiness (การร้องของเพลง)  ค่อนข้างเยอะ

ประเภทที่ 3

Beats.Per.Minute (bpm) มีจังหวะที่เร็ว
Energy (ความแรง) ปานกลาง
Danceability (อัตราการเต้น) ปานกลาง
Loudness (ความดังในหน่วย dB) ปานกลาง
Liveness (ความสดของดนตรี)มีค่าปกติ
Valence (ความจุเพลง) น้อย
Duration (ความยาวเพลง ) อยู่ที่ประมาณ 4.3 นาที
Acousticness (อารมณ์ของเพลง) เป็นเพลงที่มีทั้งร้องและแร๊ปค่อนข้างเยอะ
Speechiness (การร้องของเพลง)  ค่อนข้างเยอะ

ประเภทที่ 4

Beats.Per.Minute (bpm) มีจังหวะที่เร็ว
Energy (ความแรง) น้อยมาก
Danceability (อัตราการเต้น) น้อย
Loudness (ความดังในหน่วย dB) ค่อนข้างเบา
Liveness (ความสดของดนตรี)มีค่าปกติ
Valence (ความจุเพลง) น้อยมาก
Duration (ความยาวเพลง) อยู่ที่ประมาณ 4.3 นาที
Acousticness (อารมณ์ของเพลง) เป็นเพลงที่สนุกสนาน
Speechiness (การร้องของเพลง)  ค่อนข้างน้อย

ประเภทที่ 5

Beats.Per.Minute (bpm) มีจังหวะที่เร็ว
Energy (ความแรง) ปานกลาง
Danceability (อัตราการเต้น) ปานกลาง
Loudness (ความดังในหน่วย dB) ปานกลาง
Liveness (ความสดของดนตรี) มีค่าปกติ
Valence (ความจุเพลง) น้อยมาก
Duration (ความยาวเพลง) อยู่ที่ประมาณ 4.3 นาที
Acousticness (อารมณ์ของเพลง) เป็นเพลงที่ร้องปกติ
Speechiness (การร้องของเพลง)  ปกติ

ประเภทที่ 6

Beats.Per.Minute (bpm) มีจังหวะปกติ
Energy (ความแรง) ปานกลาง
Danceability (อัตราการเต้น) ปานกลาง
Loudness (ความดังในหน่วย dB) ปานกลาง
Liveness (ความสดของดนตรี) มีค่าน้อย
Valence (ความจุเพลง) น้อยมาก
Duration (ความยาวเพลง) อยู่ที่ประมาณ 3.6 นาที
Acousticness (อารมณ์ของเพลง) เป็นเพลงที่ร้องปกติ
Speechiness (การร้องของเพลง)  ปกติ

ประเภทที่ 7

Beats.Per.Minute (bpm) มีจังหวะปกติ
Energy (ความแรง) น้อย
Danceability (อัตราการเต้น) ปานกลาง
Loudness (ความดังในหน่วย dB) เบา
Liveness (ความสดของดนตรี) มีค่าน้อย
Valence (ความจุเพลง) น้อย
Duration (ความยาวเพลง) อยู่ที่ประมาณ 3.2 นาที
Acousticness (อารมณ์ของเพลง) ค่อนข้างสนุกสนาน
Speechiness (การร้องของเพลง)  การร้องค่อนข้างน้อย

ประเภทที่ 8

Beats.Per.Minute (bpm) มีจังหวะปกติ
Energy (ความแรง) ปานกลาง
Danceability (อัตราการเต้น) ปานกลาง
Loudness (ความดังในหน่วย dB) ปกติ
Liveness (ความสดของดนตรี) ค่อนข้างเยอะ
Valence (ความจุเพลง) ค่อนข้างน้อย
Duration (ความยาวเพลง) อยู่ที่ประมาณ 5.6 นาที
Acousticness (อารมณ์ของเพลง) เป็นเพลงที่ร้องปกติ
Speechiness (การร้องของเพลง)  มีการร้องค่อนข้างเยอะ

ประเภทที่ 9

Beats.Per.Minute (bpm) มีจังหวะที่เร็ว
Energy (ความแรง) ปานกลาง
Danceability (อัตราการเต้น) ปานกลาง
Loudness (ความดังในหน่วย dB) ปานกลาง
Liveness (ความสดของดนตรี)มีค่าปกติ
Valence (ความจุเพลง) น้อย
Duration (ความยาวเพลง ) อยู่ที่ประมาณ 3.1 นาที
Acousticness (อารมณ์ของเพลง) เป็นเพลงที่มีทั้งร้องและแร๊ปค่อนข้างเยอะ
Speechiness (การร้องของเพลง)  ค่อนข้างเยอะ

* **ซึ่งจากการสรุปทั้งหมดแล้วนั้น สรุปได้ว่าถ้านักแต่งเพลงต้องการจะแต่งเพลงใดเพลงหนึ่งขึ้นมา เราก็จะทำการเทรนข้อมูล และได้ออกมาเป็น 10 กลุ่ม แต่บางกลุ่มนั้นมีผลใกล้เคียงมาก เราจึงสรุปออกมาเป็น 9 ประเภทหรือ 9 กลุ่มที่มีความแตกต่างกันบางอย่าง เพื่อเป็นตัวเลือกหรือแนวทางในการแต่งเพลงและได้ศึกษาข้อมูลต่างๆเหล่านี้** 

และที่ได้แบ่งเป็น 10 ประเภทเพราะนักแต่งเพลงต้องการว่าประเภทของข้อมูลที่หลากหลาย หลายแนว และได้ทำการทดสอบแล้ว ตั้งแต่ 3-20 และพบว่า การทำเป็น 10 กลุ่มดีที่สุด เพราะ ได้ข้อมูลหลากหลาย ข้อมูลซ้ำกันน้อย มีค่าที่แตกต่างกันอย่างเห็นได้ชัดเจน